# 大作业part1：python爬虫
> 张驰 zhang-ch22@mails.tsinghua.edu.cn Isomorphic

In [ ]:
import json
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wdw
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import selenium
from time import sleep
options = selenium.webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0')
prefs = {
    'profile.default_content_setting_values':{
        'images':2,
    }
}
options.add_experimental_option("excludeSwitches", ['enable-automation'])
# options.add_experimental_option('prefs',prefs)
# options.add_argument("--disable-plugins")
driver = selenium.webdriver.Chrome(options = options)
print("Done")

首先爬取首页，获得新闻正文页链接，存放到links.csv中

In [ ]:
with open("links.csv",'w',encoding='utf-8') as file:
    file.write("title,link\n")
    for page in range(1,51):
        print("current page:",page)
        driver.get(f"https://news.sina.com.cn/roll/#pageid=153&lid=2515&k=&num=50&page={page}")
        driver.refresh()
        driver.execute_script("window.scrollBy(0, 1);")
        spans = driver.find_elements(By.CLASS_NAME,'c_tit')
        count = 0
        for span in spans:
            count+=1
            try:
                site = span.find_element(By.TAG_NAME,'a')
                link = site.get_attribute('href')
                title = site.text
                file.write(title+','+link+'\n')
            except StaleElementReferenceException:
                print("Exception:",page,count)
                continue

print("Done")

这些为2023.8.28的新闻，不够5000条。下面爬取过去一年的新闻。2022.8.28-2023.8.27


In [ ]:
from datetime import *
#2022-08-28
etime0 = 1670428800
stime0 = 1670515200
date0 = datetime(2022,8,28).date()

with open("links1year.csv",'a',encoding='utf-8') as file:

    for days in range(-1396,-10000,-1):

        if days%100==0:
            driver.close()
            options = selenium.webdriver.ChromeOptions()
            options.add_argument('User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0')
           
            options.add_experimental_option("excludeSwitches", ['enable-automation'])

            driver = selenium.webdriver.Chrome(options = options)

        etime = etime0 + 86400*days
        stime = stime0 + 86400*days
        date = date0 + timedelta(days=days)
        datestr = date.strftime('%Y-%m-%d')
        print("current date:",datestr)
        page=0
        while True:
            page+=1
            url = f"https://news.sina.com.cn/roll/#pageid=153&lid=2515&etime={etime}&stime={stime}&ctime={stime}&date={datestr}&k=&num=50&page={page}"
            
            driver.get(url)
            driver.refresh()
            driver.execute_script("window.scrollBy(0, 1);")
            spans = driver.find_elements(By.CLASS_NAME,'c_tit')
            count = 0
            for span in spans:
                count+=1
                try:
                    site = span.find_element(By.TAG_NAME,'a')
                    link = site.get_attribute('href')
                    title = site.text
                    file.write(title+','+link+'\n')
                except:
                    print("Exception:",page,count)
                    continue
            if count<50:break

print("Done")
with open("links1year.csv",'r',encoding='utf-8') as infile, open("links_processed.csv",'w',encoding='utf-8') as outfile:
    for line in infile:
        line = line.strip()
        index = line.rfind(",")
        outline = f'"{line[:index]}","{line[index+1:]}"'
        outfile.write(outline+'\n')

储存的csv文件有的行自带“,”导致分块出现问题，下统一处理

之后再读取links_processed.csv中的网址，爬取新闻正文


In [ ]:
import pandas as pd
def extract_text(p_elem):
    try:
        child_elem = p_elem.find_element(By.CSS_SELECTOR,'*')
        return extract_text(child_elem)
    except NoSuchElementException:
        return p_elem.text
        
# df = pd.read_csv("links.csv",encoding='utf-8')
df = pd.read_csv("links_processed.csv",encoding='utf-8')
links = df["link"].values #python list
titles = df["title"].values
N = df.shape[0]
for i in range(220,N):# 此处从249开始

    if i%100==0:
        driver.close()
        options = selenium.webdriver.ChromeOptions()
        options.add_argument('User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0')
        prefs = {
            'profile.default_content_setting_values':{
                'images':2,
                'css':2
            }
        }
        options.add_experimental_option("excludeSwitches", ['enable-automation'])
        # options.add_experimental_option('prefs',prefs)
        # options.add_argument("--disable-plugins")
        driver = selenium.webdriver.Chrome(options = options)
    print(i)
    link = links[i]
    driver.get(link)
    driver.refresh()

    author = "NaN"
    publish_time = "NaN"
    try:
        publish_time = extract_text(driver.find_element(By.CLASS_NAME,'date'))
    except NoSuchElementException:
        try: publish_time = extract_text(driver.find_element(By.ID,'pub_date'))
        except: pass
    try:
        author = extract_text(driver.find_element(By.CLASS_NAME,'source'))
    except NoSuchElementException:
        try: 
            author = extract_text(driver.find_element(By.CLASS_NAME,'author'))
        except NoSuchElementException: 
            try: author = author = extract_text(driver.find_element(By.ID,'media_name'))
            except: pass

    article = []
    try:
        artibody = driver.find_element(By.ID,'artibody')
        paras = artibody.find_elements(By.CSS_SELECTOR,'*')
        for para in paras:
            if para.tag_name=='p':
                article.append(para.text)
            else:
                try:
                    img = para.find_element(By.TAG_NAME,'img')
                    img_link = img.get_attribute("src")
                    img_text = img.get_attribute("alt")
                    article.append([img_link,img_text])
                except: pass
    except: pass

    comments = []
    try: 
        comment_page = driver.find_element(By.CLASS_NAME,'sina-comment-page')
        heads = comment_page.find_elements(By.CLASS_NAME,'head')
        contents = comment_page.find_elements(By.CLASS_NAME,'cont')
        for c,h in zip(contents,heads):
            area = "NaN"
            try: area = c.find_element(By.CLASS_NAME,'area').text
            except: pass
            text = "NaN"
            try: text = c.find_element(By.CLASS_NAME,'txt').text
            except: pass
            head_img = "NaN"
            try: head_img = h.find_element(By.TAG_NAME,'img').get_attribute('src')
            except: pass
            time = "NaN"
            try: time = c.find_element(By.CLASS_NAME,'time').text
            except: pass
            name = "NaN"
            try: name = h.find_element(By.TAG_NAME,'a').get_attribute('title')
            except: pass
            comments.append([name,head_img,area,text,time])
    except: pass

    comment_count = 'NaN'
    comment_users = 'NaN' #参与用户数
    try: 
        count = driver.find_element(By.CLASS_NAME,'count')
        twoelems = count.find_elements(By.TAG_NAME,'a')
        print(len(twoelems))
        if len(twoelems)==2:
            comment_count = twoelems[0].text
            comment_users = twoelems[1].text
    except: pass

    title = titles[i]
    link = links[i]
    data = {"title":title,"pub_date":publish_time,"author":author,"article":article,"comment_count":comment_count,"comment_users":comment_users,"comments":comments,"link":link}
    df = pd.DataFrame(data.items())
    df.to_csv(f"./results/{i+1}.csv",index=False,encoding='utf-8')
    

爬取的结果都在"./results"里，每个网页有一个独立的.csv文件

由于selenium太慢，下面不用selenium重写了一遍

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
from time import sleep
from fake_useragent import UserAgent


def extract_text(p_elem):
    if p_elem.find() is not None:
        return extract_text(p_elem.find())
    else:
        return p_elem.text

df = pd.read_csv("links_processed.csv",encoding='utf-8')
links = df["link"].values #python list
titles = df["title"].values
N = df.shape[0]

for i in range(N):# 此处从249开始
    if i%11==5: sleep(1)
    if i%37==6: sleep(2)
    if i%97==7: sleep(5)

    print(i)
    link = links[i]
    ua=UserAgent()
    headers={"User-Agent":ua.random} 
    response = requests.get(link,headers=headers)
    sleep(0.1)
    response.encoding = 'UTF-8'
    soup = BeautifulSoup(response.text,'lxml')

    author = ""

    a = soup.find(class_='source')
    if a is not None:
        author = extract_text(a)
    else:
        b = soup.find(class_='author')
        if b is not None:
            author = extract_text(b)
        else:
            c = soup.find(id='media_name')
            if c is not None:
                author = extract_text(c)

    article = []
    artibody = soup.find(id='artibody')
    if artibody is not None:
        paras = artibody.find_all()
        for para in paras:
            if para.name=='p':
                article.append(para.text)
            elif 'class' in para.attrs:
                if para.attrs['class']==['img_wrapper']:
                    img = para.find('img')
                    
                    if img is not None:
                        img_link = ""
                        img_text = ""
                        if 'src' in img.attrs:
                            img_link = "https:"+img.attrs["src"]
                        if 'alt' in img.attrs:
                            img_text = img.attrs["alt"]
                        article.append([img_link,img_text])

    match = re.search(r"doc-i(.+?)\.shtml",link)
    newsid = match.group(1)
    comment_url_kj = f"https://comment5.news.sina.com.cn/page/info?version=1&format=json&channel=kj&newsid=comos-{newsid}&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&h_size=3&thread=1"
    comment_url_cj = f"https://comment5.news.sina.com.cn/page/info?version=1&format=json&channel=cj&newsid=comos-{newsid}&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&h_size=3&thread=1"

    comment_count_cj = 0
    comment_count_kj = 0
    comment_users_cj = 0
    comment_users_kj = 0

    response_cj = requests.get(comment_url_cj)
    response_kj = requests.get(comment_url_kj)
    sleep(0.1)

    response_cj.encoding = 'UTF-8'
    cmtpg_cj = response_cj.text
    jd_cj = json.loads(cmtpg_cj)
    if 'count' in jd_cj['result']:
        cntcj = jd_cj['result']['count']
        comment_count_cj = cntcj['show']
        comment_users_cj = cntcj['total']

    response_kj.encoding = 'UTF-8'
    cmtpg_kj = response_kj.text
    jd_kj = json.loads(cmtpg_kj)
    if 'count' in jd_kj['result']:
        cntkj = jd_kj['result']['count']
        comment_count_kj = cntkj['show']
        comment_users_kj = cntkj['total'] 

    comment_count = max(comment_count_cj,comment_count_kj)
    comment_users = max(comment_users_cj,comment_users_kj)

    pub_date = ""
    if 'news' in jd_cj['result']:
        pub_date = jd_cj['result']['news']['time']
    if pub_date=='' and 'news' in jd_kj['result']:
        pub_date = jd_kj['result']['news']['time']

    title = titles[i]
    link = links[i]
    data = {"title":title,"pub_date":pub_date,"author":author,"article":article,"comment_count":comment_count,"comment_users":comment_users,"link":link}
    df = pd.DataFrame(data.items())
    df.to_csv(f"./results/{i+1}.csv",index=False,encoding='utf-8')